In [1]:
import numpy as np
from qore import Mine, ASP, QAOA, VQE, Pseudoflow, PseudoflowSolver, ASPSolver

In [7]:
qmine = Mine(np.array([[-2., 3., 1.], [float('inf'), 5., float('inf')]]))
qmine.plot_mine()

+---+--------+-------+-------+
|   |   0    |   1   |   2   |
+---+--------+-------+-------+
| 0 | -2.000 | 3.000 | 1.000 |
| 1 |  inf   | 5.000 |  inf  |
+---+--------+-------+-------+


In [3]:
pf = PseudoflowSolver(qmine)

In [4]:
res = pf.solve()

In [5]:
res

{'opt_config': '1111'}

In [8]:
asp = ASPSolver(qmine)

In [9]:
penalty = 10.0
evol_time = 20
nsteps = 50

In [11]:
res = asp.solve(penalty=penalty, evol_time=evol_time, nsteps=nsteps)

/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/aqua/operators/legacy/weighted_pauli_operator.py:906: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc = QuantumCircuit() + state_in
/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/circuit/quantumcircuit.py:869: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)


In [12]:
res

{'opt_config': '1111',
 'opt_config_prob': 0.9287109375,
 'profit_avg': 6.753832133010106,
 'violation_avg': 8.87761596793435e-05,
 'ground_state': {'0111': 0.0048828125,
  '0010': 0.0009765625,
  '0110': 0.0654296875,
  '1111': 0.9287109375}}